In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./example-analytics-data.csv", index_col=None, parse_dates=['Date of Birth'])

In [3]:
df.head()

,User ID,Name,Date of Birth,Session ID,Date,Page URL,Pageviews,Session Duration,Bounce Rate,Device Type,Traffic Source,Location,Conversion
0,1,John Doe,2004-03-06,S001,2024-11-25,/home,5,320,20,Mobile,Organic Search,"New York, USA",Yes
1,1,John Doe,2004-03-06,S008,2024-11-26,/product/xyz,4,200,25,Desktop,Direct,"New York, USA",No
2,2,Jane Doe,2002-06-10,S002,2024-11-25,/product/abc,3,180,33,Desktop,Direct,"London, UK",No
3,2,Jane Doe,2002-06-10,S009,2024-11-26,/checkout,7,350,14,Tablet,Paid Search,"London, UK",Yes
4,3,Timothy James,1999-11-03,S003,2024-11-26,/blog/5-tips,2,90,50,Mobile,Social Media,"Sydney, AUS",No


In [4]:
df.dtypes

User ID                      int64
Name                        object
Date of Birth       datetime64[ns]
Session ID                  object
Date                        object
Page URL                    object
Pageviews                    int64
Session Duration             int64
Bounce Rate                  int64
Device Type                 object
Traffic Source              object
Location                    object
Conversion                  object
dtype: object

## Prompt

Imagine you want to send the above analytics information (that came from somewhere like your internal Google Analytics platform) to ChatGPT to be analyzed to spot trends and forecast traffic.

1. What PII is in here that needs to be protected?
2. How could you mask that PII so that this data could be passed to ChatGPT?

*Fun fact:* ChatGPT generated most of this CSV, but it forgot to enclose the `Location` field in quotes, which threw me off for a long time.

In [5]:
df['age'] = df['Date of Birth'].apply(lambda x: 2024 - x.year)

In [6]:
df.head()

,User ID,Name,Date of Birth,Session ID,Date,Page URL,Pageviews,Session Duration,Bounce Rate,Device Type,Traffic Source,Location,Conversion,age
0,1,John Doe,2004-03-06,S001,2024-11-25,/home,5,320,20,Mobile,Organic Search,"New York, USA",Yes,20
1,1,John Doe,2004-03-06,S008,2024-11-26,/product/xyz,4,200,25,Desktop,Direct,"New York, USA",No,20
2,2,Jane Doe,2002-06-10,S002,2024-11-25,/product/abc,3,180,33,Desktop,Direct,"London, UK",No,22
3,2,Jane Doe,2002-06-10,S009,2024-11-26,/checkout,7,350,14,Tablet,Paid Search,"London, UK",Yes,22
4,3,Timothy James,1999-11-03,S003,2024-11-26,/blog/5-tips,2,90,50,Mobile,Social Media,"Sydney, AUS",No,25


In [21]:
city_to_code = {
    "New York, USA": 0,
    "London, UK": 1,
    "Sydney, AUS": 2,
    "Mumbai, IND": 3,
    "Toronto, CAN": 4,
    "Berlin, DEU": 5,
    "Paris, FRA": 6
}

In [22]:
df['location_code'] = df['Location'].apply(lambda x: city_to_code.get(x))

In [23]:
pii_columns = ['Name', 'Date of Birth', 'Location']

In [24]:
columns_to_use = [ c for c in df.columns if c not in pii_columns ]

In [25]:
df_clean = df[columns_to_use].copy()

In [26]:
df_clean.head()

,User ID,Session ID,Date,Page URL,Pageviews,Session Duration,Bounce Rate,Device Type,Traffic Source,Conversion,age,location_code
0,1,S001,2024-11-25,/home,5,320,20,Mobile,Organic Search,Yes,20,0
1,1,S008,2024-11-26,/product/xyz,4,200,25,Desktop,Direct,No,20,0
2,2,S002,2024-11-25,/product/abc,3,180,33,Desktop,Direct,No,22,1
3,2,S009,2024-11-26,/checkout,7,350,14,Tablet,Paid Search,Yes,22,1
4,3,S003,2024-11-26,/blog/5-tips,2,90,50,Mobile,Social Media,No,25,2


In [27]:
df_clean.to_csv('./clean-example-analytics-data.csv')